# LangChain Expression Language (LCEL)

LangChain Expression Language (LCEL)은 체인을 쉽게 함께 구성할 수 있는 선언적 방법을 제공합니다. LCEL은 처음부터 프로토타입을 생산 환경에 코드 변경 없이 적용할 수 있도록 지원하도록 설계되었습니다. 가장 단순한 “프롬프트 + LLM” 체인부터 생산 환경에서 수백 단계의 LCEL 체인을 성공적으로 실행한 복잡한 체인까지 모두 지원합니다. LCEL을 사용하고 싶을 수 있는 몇 가지 이유를 강조해 보겠습니다:

스트리밍 지원: LCEL로 체인을 구축하면 최적의 첫 번째 토큰까지의 시간(출력의 첫 청크가 나올 때까지의 시간)을 얻을 수 있습니다. 일부 체인의 경우, 예를 들어 LLM에서 스트리밍 출력 파서로 토큰을 직접 스트리밍하고, LLM 제공자가 원시 토큰을 출력하는 속도로 파싱된 증분 청크의 출력을 받을 수 있습니다.

비동기 지원: LCEL로 구축된 모든 체인은 동기 API(예: 프로토타이핑하는 동안의 주피터 노트북에서)와 비동기 API(예: LangServe 서버에서) 모두에서 호출할 수 있습니다. 이를 통해 프로토타입과 생산 환경에서 동일한 코드를 사용하며, 뛰어난 성능과 동시에 많은 동시 요청을 동일한 서버에서 처리할 수 있는 능력을 제공합니다.

최적화된 병렬 실행: LCEL 체인에 병렬로 실행할 수 있는 단계가 있을 때마다(예: 여러 검색기에서 문서를 가져올 때) 동기 및 비동기 인터페이스 모두에서 자동으로 수행하여 가능한 한 지연 시간을 최소화합니다.

재시도 및 대체 구성: LCEL 체인의 모든 부분에 대해 재시도 및 대체를 구성할 수 있습니다. 이는 체인을 대규모로 더 신뢰할 수 있게 만드는 훌륭한 방법입니다. 현재 재시도/대체에 대한 스트리밍 지원을 추가하는 작업을 진행 중이며, 이를 통해 어떠한 지연 시간 비용 없이 추가적인 신뢰성을 얻을 수 있습니다.

중간 결과 접근: 더 복잡한 체인의 경우, 최종 출력이 생성되기 전에 중간 단계의 결과에 접근하는 것이 매우 유용할 수 있습니다. 이를 통해 최종 사용자에게 무언가가 진행되고 있음을 알리거나, 체인을 디버깅하는 데 사용할 수 있습니다. 중간 결과를 스트리밍할 수 있으며, 모든 LangServe 서버에서 사용할 수 있습니다.

입력 및 출력 스키마: 입력 및 출력 스키마는 모든 LCEL 체인에 Pydantic 및 JSONSchema 스키마를 체인의 구조에서 추론하여 제공합니다. 이는 입력과 출력의 유효성 검사에 사용될 수 있으며 LangServe의 중요한 부분입니다.

LangSmith 추적 통합: 체인이 점점 더 복잡해짐에 따라, 모든 단계에서 정확히 무슨 일이 일어나고 있는지 이해하는 것이 점점 더 중요해집니다. LCEL을 사용하면, 모든 단계가 LangSmith에 자동으로 기록되어 최대한의 관찰 가능성과 디버깅 가능성을 제공합니다.

LangServe 배포 통합: LCEL로 생성된 모든 체인은 LangServe를 사용하여 쉽게 배포할 수 있습니다. 이는 개발자가 복잡한 배포 과정 없이도 빠르고 효율적으로 애플리케이션을 생산 환경에 배포할 수 있게 해줍니다.
LCEL은 개발자가 더 빠르고, 더 안정적이며, 더 유연한 방식으로 체인을 구축하고 배포할 수 있도록 설계되었습니다. 이를 통해 프로토타이핑에서부터 생산 환경까지 일관된 개발 경험을 제공하며, 애플리케이션의 확장성과 신뢰성을 향상시키는 동시에 개발 및 운영 비용을 절감할 수 있습니다. 따라서, 복잡한 로직을 포함한 애플리케이션을 개발하거나, 다양한 외부 리소스와의 통합이 필요한 프로젝트에 LCEL을 활용하는 것을 고려해 볼 수 있습니다.

LCEL은 기본 구성 요소에서 복잡한 체인을 쉽게 구축할 수 있도록 해주며, 스트리밍, 병렬 처리, 로깅과 같은 기능을 상자에서 꺼내자마자 지원합니다.

기본 예제: 프롬프트 + 모델 + 출력 파서
가장 기본적이고 일반적인 사용 사례는 프롬프트 템플릿과 모델을 함께 체이닝하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 주제를 받아서 농담을 생성하는 체인을 만들어 보겠습니다:

In [1]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import HumanMessage


prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they're always getting smashed!"

이 코드의 결과로, "아이스크림이 파티에 절대 초대받지 못하는 이유는 뭘까요? 열이 나면 항상 녹아내리기 때문이죠!"와 같은 농담을 생성할 수 있습니다.

이 코드에서 다양한 구성 요소를 LCEL을 사용하여 단일 체인으로 결합하는 이 줄을 주목하세요:

In [3]:
chain = prompt | model | output_parser

`|` 기호는 유닉스 파이프 연산자와 유사하며, 다양한 구성 요소를 체이닝하여 하나의 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿의 출력은 모델로 전달되며, 그 다음 모델의 출력은 출력 파서로 전달됩니다. 각 구성 요소를 개별적으로 살펴봄으로써 정말로 무슨 일이 일어나고 있는지 이해해 보겠습니다.

### Prompt

`prompt`는 `BasePromptTemplate`의 인스턴스로, 템플릿 변수의 딕셔너리를 입력으로 받아 `PromptValue`를 생성합니다.

`PromptValue`는 완성된 프롬프트를 래핑한 것으로, 문자열을 입력으로 받는 LLM(대형 언어 모델)이나 메시지 시퀀스를 입력으로 받는 `ChatModel`에 전달될 수 있습니다.

BaseMessages`를 생성하는 로직과 문자열을 생성하는 로직을 모두 정의하기 때문에, 언어 모델 유형에 상관없이 작동할 수 있습니다.

In [4]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

이 코드는 "ice cream"에 대한 짧은 농담을 요청하는 `ChatPromptValue`를 생성합니다. 이 값은 다음과 같이 `messages` 리스트를 포함하며, 이 리스트 안에는 `HumanMessage` 객체가 있습니다:

In [8]:
ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

`prompt_value`에서 `to_messages()` 메소드를 호출하면, 프롬프트를 `HumanMessage` 객체 리스트로 변환할 수 있습니다. 이는 체인의 다음 단계로 메시지를 전달하는 데 사용될 수 있습니다:

In [9]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

출력 결과:

In [10]:
[HumanMessage(content='tell me a short joke about ice cream')]

[HumanMessage(content='tell me a short joke about ice cream')]

또한, `prompt_value`에서 `to_string()` 메소드를 호출하면, 프롬프트를 단순한 문자열로 변환할 수 있으며, 이 문자열은 LLM 같은 다른 모델에 입력으로 사용될 수 있습니다:

In [11]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

출력 결과:

In [ ]:
'Human: tell me a short joke about ice cream'

이렇게, `prompt`는 다양한 언어 모델에 적합한 입력을 생성할 수 있는 유연하고 효율적인 방법을 제공합니다.

### Model

`PromptValue`가 모델에 전달된 다음 단계입니다. 이 경우 우리의 모델은 `ChatModel`이며, 이는 `BaseMessage`를 출력합니다.

In [12]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when things heat up!", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

이 코드는 AI로부터 "왜 아이스크림은 파티에 절대 초대받지 않나요?\n\n왜냐하면 그들은 항상 멜트다운을 가져오기 때문이죠!" 같은 내용의 AIMessage를 생성합니다.

만약 우리의 모델이 LLM이었다면, 문자열을 출력했을 것입니다.

In [13]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why did the ice cream go to therapy? Because it was feeling a little rocky road!'

이 경우, LLM은 "아이스크림 트럭이 고장난 이유는 무엇인가요? 멜트다운 때문이죠!"와 같은 문자열을 출력합니다.

이 예시에서 볼 수 있듯이, model.invoke() 메소드는 입력된 PromptValue를 바탕으로 모델에 의해 생성된 응답을 반환합니다. ChatModel을 사용하는 경우, 이 응답은 대화의 일부인 BaseMessage 형태로, LLM을 사용하는 경우에는 단순한 문자열 형태로 반환됩니다. 이는 LCEL 체인에서 다음 구성 요소로 쉽게 전달할 수 있는 유연하고 일관된 방법을 제공합니다.

### Output parser


마지막으로, 모델의 출력을 `output_parser`에 전달합니다. 

여기서 `output_parser`는 `BaseOutputParser`의 인스턴스로, 문자열이나 `BaseMessage`를 입력으로 받습니다. 

특히 `StrOutputParser`는 입력된 어떤 것이든 간단히 문자열로 변환합니다.

In [14]:
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always drip when things heat up!"

이 코드는 "아이스크림이 치료를 받으러 간 이유는 무엇인가요? \n\n너무 많은 토핑을 가지고 있어서 자신의 '콘-피던스'를 찾을 수 없었기 때문이죠!"와 같은 문자열을 생성합니다.

이 예제에서, `output_parser`는 모델로부터 받은 출력`('message')`을 처리하여 최종적인 문자열 형태의 응답을 생성합니다. 이렇게 `StrOutputParser`는 모델의 출력이 `BaseMessage` 형태이든, 이미 문자열 형태이든 상관없이, 일관되게 문자열로 변환하는 역할을 합니다. 이 과정은 LCEL 체인의 마지막 단계로서, 최종 사용자에게 보여줄 결과를 준비하는 중요한 작업입니다.

### Entire Pipeline

전체 파이프라인을 따라가 보면 다음과 같은 단계를 거칩니다:

1. 사용자 입력을 주제에 대한 요청과 함께 전달합니다. 예: {"topic": "ice cream"}

2. 프롬프트 구성 요소는 사용자 입력을 받아 주제를 사용하여 프롬프트를 구성한 후 PromptValue를 생성합니다.

3. 모델 구성 요소는 생성된 프롬프트를 받아 OpenAI LLM 모델로 전달하여 평가합니다. 모델에서 생성된 출력은 ChatMessage 객체입니다.

4. 마지막으로, output_parser 구성 요소는 ChatMessage를 받아 이를 파이썬 문자열로 변환하고, 이 문자열은 invoke 메소드에서 반환됩니다.

이러한 단계를 통해, 사용자로부터 특정 주제에 대한 요청을 받고, 이를 기반으로 프롬프트를 생성하며, 이 프롬프트를 모델에 전달하여 응답을 생성하고, 최종적으로 이 응답을 사용자가 이해할 수 있는 문자열 형태로 변환하는 전체 과정을 완성합니다. 

이 파이프라인은 LCEL을 사용하여 구성되어 있으며, 각 구성 요소는 특정 기능을 수행하여 전체적인 작업 흐름을 원활하게 만듭니다.


구성 요소의 출력이 궁금한 경우, prompt나 prompt | model과 같이 체인의 일부분만 테스트하여 중간 결과를 확인할 수 있습니다. 이를 통해 각 단계에서 어떤 일이 일어나는지 더 잘 이해하고, 최종 결과에 어떻게 기여하는지 볼 수 있습니다.

예를 들어, 단순히 프롬프트 구성 요소만을 테스트하고 싶다면 다음과 같이 할 수 있습니다:

In [15]:
input = {"topic": "ice cream"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

(prompt | model).invoke(input)
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always melt under pressure!", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 15, 'total_tokens': 33}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

### RAG Search Example

이번 예제에서는, 질문에 답변할 때 일부 맥락을 추가하기 위해 검색-증강 생성(Retrieval-Augmented Generation, RAG) 체인을 실행하려고 합니다.

이 체인은 사용자의 질문에 기반한 맥락 정보를 검색하여, 그 맥락을 바탕으로 질문에 대한 답변을 생성합니다. 이 과정은 다음과 같은 구성 요소들로 이루어져 있습니다:

1. Vectorstore와 Retriever: 문서(또는 맥락 정보) 집합에서 관련 정보를 검색합니다. 이 예제에서는 DocArrayInMemorySearch를 사용하여 메모리 내에서 텍스트의 벡터 표현을 저장하고 검색하는 vectorstore를 만들고, 이를 retriever로 사용합니다.

2. Prompt Template: 검색된 맥락과 사용자의 질문을 포함하는 프롬프트를 생성합니다. 여기서는 질문에 기반한 맥락 정보를 포함시키기 위해 템플릿을 사용합니다.

3. Model: 생성된 프롬프트를 모델(예: ChatOpenAI)에 전달하여 답변을 생성합니다.

4. Output Parser: 모델의 출력을 처리하여 최종 답변을 문자열로 변환합니다.

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})

이 코드를 실행하면, 사용자의 질문 "where did harrison work?"에 대해 관련 맥락 "harrison worked at kensho"를 검색하여, 이 맥락을 바탕으로 질문에 답변하는 체인이 작동합니다. 이를 통해 답변의 정확도와 관련성을 높일 수 있습니다.

이 경우에 구성된 체인은 다음과 같습니다:

In [ ]:
chain = setup_and_retrieval | prompt | model | output_parser

이를 설명하기 위해, 위의 프롬프트 템플릿이 맥락(context)과 질문(question)을 프롬프트에 대체할 값으로 받아들인다는 것을 알 수 있습니다. 프롬프트 템플릿을 구축하기 전에, 우리는 검색에 관련된 문서를 검색하여 맥락의 일부로 포함시키고자 합니다.

예비 단계로, 우리는 쿼리에 기반해 문서를 검색할 수 있는 메모리 내 저장소를 사용하여 검색기를 설정했습니다. 이것은 다른 구성 요소들과 함께 체인될 수 있는 실행 가능한 구성 요소이지만, 별도로 실행해 보는 것도 가능합니다:

In [ ]:
retriever.invoke("where did harrison work?")

그 다음, 검색된 문서의 항목과 원본 사용자 질문을 사용하여 프롬프트에 필요한 입력을 준비하기 위해 RunnableParallel을 사용합니다. 여기서 문서 검색을 위한 retriever와 사용자의 질문을 전달하기 위한 RunnablePassthrough를 사용합니다:

In [ ]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

전체 체인을 복습해 보면:

In [ ]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

흐름은 다음과 같습니다:

1. 첫 번째 단계에서 RunnableParallel 객체를 두 개의 항목과 함께 생성합니다. 첫 번째 항목인 context는 검색기에 의해 가져온 문서 결과를 포함할 것입니다. 두 번째 항목인 question은 사용자의 원래 질문을 포함할 것입니다. 이 질문을 전달하기 위해 RunnablePassthrough를 사용하여 이 항목을 복사합니다.

2. 위 단계에서의 딕셔너리를 프롬프트 구성 요소에 전달합니다. 그러면 사용자 입력인 question과 검색된 문서인 context를 사용하여 프롬프트를 구성하고 PromptValue를 출력합니다.

3. 모델 구성 요소는 생성된 프롬프트를 가져와 OpenAI LLM 모델로 평가를 위해 전달합니다. 모델에서 생성된 출력은 ChatMessage 객체입니다.

4. 마지막으로, output_parser 구성 요소는 ChatMessage를 받아 이를 파이썬 문자열로 변환하며, 이 문자열은 invoke 메소드에서 반환됩니다.

# Why use LCEL

LCEL(LangChain Expression Language)은 기본 구성 요소에서 복잡한 체인을 쉽게 구축할 수 있도록 합니다. 이는 다음을 제공함으로써 이루어집니다:

1. 통합 인터페이스: 모든 LCEL 객체는 Runnable 인터페이스를 구현합니다. 이 인터페이스는 일반적인 일련의 호출 메소드(invoke, batch, stream, ainvoke, 등)를 정의합니다. 이를 통해 LCEL 객체의 체인도 이러한 호출을 자동으로 지원할 수 있습니다. 즉, LCEL 객체의 모든 체인은 그 자체로 LCEL 객체입니다.

2. 구성 기본 요소: LCEL은 체인을 구성하고, 구성 요소를 병렬로 실행하며, 대체를 추가하고, 체인 내부를 동적으로 구성하는 등을 쉽게 할 수 있는 다양한 기본 요소를 제공합니다.
LCEL의 가치를 더 잘 이해하기 위해서는 이를 실제로 사용해 보고, 이를 없이 유사한 기능을 재현하려면 어떻게 해야 할지 생각해 보는 것이 도움이 됩니다. 이 워크스루에서는 바로 그렇게 할 것입니다. 우리는 '시작하기' 섹션에서의 기본 예제인 간단한 프롬프트 + 

모델 체인을 취하고, 그 아래에서 이미 많은 기능을 정의하고 있는 것을 볼 것이며, 그것을 재현하기 위해 필요한 것이 무엇인지 살펴볼 것입니다.

In [ ]:
%pip install --upgrade --quiet langchain-core langchain-openai langchain-anthropic

이제 필요한 라이브러리를 설치했으니, 간단한 프롬프트 + 모델 체인을 구현해 보겠습니다. 우리는 이미 `prompt`, `model`, `output_parser` 구성 요소를 어떻게 사용하는지 봤습니다. 이번에는 LCEL 없이 이러한 기능을 직접 구현해 보는 것을 상상해 보세요. 

각 단계에서 필요한 로직을 직접 작성하고, 구성 요소 간의 데이터 전달을 관리하며, 비동기 실행, 스트리밍, 배치 처리와 같은 기능을 직접 구현해야 합니다. LCEL은 이 모든 것을 간단하게 만들어주며, 개발자가 더 중요한 비즈니스 로직에 집중할 수 있도록 해줍니다.

다음 두 예제는 주제 문자열을 입력으로 받아서 농담 문자열을 반환하는 가장 단순한 형태의 언어 모델 호출을 보여줍니다. LCEL을 사용하는 버전과 사용하지 않는 버전 모두 같은 결과를 내지만, 코드의 구성과 가독성에 있어서 큰 차이를 보이고 있습니다.

### invoke

In [ ]:
# without LCEL

from typing import List

import openai


prompt_template = "Tell me a short joke about {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("ice cream")

LCEL을 사용하지 않을 경우, OpenAI 클라이언트를 직접 사용하여 요청을 구성하고 실행해야 합니다.

`prompt_template`를 사용하여 주제에 따른 프롬프트를 생성하고, `call_chat_model` 함수를 통해 GPT-3.5-turbo 모델에 메시지를 전달하여 결과를 받고 있습니다. 

이 방식은 직접적이지만, 코드를 작성하고 유지 관리하는 데 있어서 세부 사항에 더 많은 주의를 기울여야 한다. 눈에 보이다 시피 그냥 복잡하기 때문입니다.

In [ ]:
# with LCEL

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

LCEL을 사용할 때는 `ChatPromptTemplate`, `StrOutputParser`, 그리고 `RunnablePassthrough`와 같은 LCEL의 구성 요소를 사용하여 체인을 구성하고 있습니다.

이러한 구성 요소들은 각각 프롬프트의 템플릿화, 모델의 출력을 문자열로 파싱, 그리고 주제 문자열을 다음 단계로 전달하는 역할을 합니다.. 체인을 구성함으로써, 각 단계가 명확하게 분리되고, 파이프라인의 각 부분이 어떻게 연결되는지 쉽게 이해할 수 있습니다.

LCEL을 사용하는 주된 이점 중 하나는 복잡한 체인을 간단하게 구성할 수 있다는 것입니다. LCEL의 구성 요소들을 사용하면, 코드의 가독성이 향상되고, 재사용 가능한 구성 요소를 통해 더 복잡한 로직을 쉽게 구현할 수 있습니다. 또한, LCEL을 사용하면 모델 호출과 관련된 로직을 추상화함으로써, 코드를 더 간결하고 관리하기 쉽게 만들 수 있습니다.

위 예제를 통해 볼 수 있듯이, LCEL을 사용하면 개발자가 언어 모델과 상호작용하는 애플리케이션을 더 쉽게, 그리고 더 효율적으로 구축할 수 있습니다. LCEL은 복잡한 처리 과정을 단순화하고, 코드의 가독성을 높여주며, 개발 과정을 가속화 합니다.

# Interface

"Runnable" 프로토콜은 사용자 정의 체인을 가능한 한 쉽게 만들 수 있도록 설계되었습니다. 대부분의 구성 요소에 대해 구현되어 있으며, 이는 표준 인터페이스를 통해 사용자 정의 체인을 정의하고 표준 방식으로 호출할 수 있게 합니다. 

이 표준 인터페이스는 다음을 포함합니다:

1. stream: 응답의 청크를 스트리밍 방식으로 반환합니다.
2. invoke: 입력에 대해 체인을 호출합니다.
3. batch: 입력 목록에 대해 체인을 호출합니다.

이러한 메소드에는 해당하는 비동기 메소드도 있어, 다양한 실행 환경과 요구 사항에 따라 유연하게 
대응할 수 있습니다:

1. astream: 응답의 청크를 비동기로 스트리밍 방식으로 반환합니다.
2. ainvoke: 입력에 대해 체인을 비동기로 호출합니다.
3. abatch: 입력 목록에 대해 체인을 비동기로 호출합니다.
3. astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍 방식으로 반환합니다. 이를 통해 개발자는 체인의 실행 과정을 더 잘 이해하고 디버깅할 수 있습니다.
5. astream_events: 체인에서 발생하는 이벤트를 실시간으로 스트리밍 방식으로 반환합니다. 이는 langchain-core 0.1.14에서 도입된 베타 기능으로, 체인의 동작을 더 세밀하게 추적할 수 있게 합니다.

이 인터페이스는 LCEL을 사용하는 개발자가 더 복잡하고 다기능적인 애플리케이션을 손쉽게 구축할 수 있도록 지원합니다. 비동기 실행 지원은 특히 현대적인 웹 애플리케이션 및 데이터 처리 워크플로우에서 중요한 역할을 하며, LCEL 체인을 사용하여 고성능 및 대응성 높은 시스템을 설계할 수 있습니다.

각 구성 요소의 입력 유형과 출력 유형은 구성 요소에 따라 다릅니다:
| Component    | Input Type                                | Output Type               |
|--------------|-------------------------------------------|---------------------------|
| Prompt       | 사전(Dictionary)                           | PromptValue               |
| ChatModel    | 단일 문자열, 채팅 메시지 목록 또는 PromptValue    | ChatMessage   |
| LLM          | 단일 문자열, 채팅 메시지 목록 또는 PromptValue    | 문자열(String)        |
| OutputParser | LLM 또는 ChatModel의 출력        | 파서에 따라 다름      |
| Retriever    | 단일 문자열                             | 문서 목록(List of Documents)         |
| Tool         | 단일 문자열 또는 도구에 따라 사전 | 도구에 따라 다름|

모든 Runnable은 입력과 출력 스키마를 노출하여 Runnable의 구조로부터 자동 생성된 입력 및 출력 Pydantic 모델을 검사할 수 있습니다:

`input_schema`: Runnable의 구조로부터 자동 생성된 입력 Pydantic 모델

`output_schema`: Runnable의 구조로부터 자동 생성된 출력 Pydantic 모델
이 메소드들을 살펴보겠습니다. 이를 위해, 매우 간단한 PromptTemplate + ChatModel 체인을 생성해 보겠습니다.

In [ ]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai

In [ ]:
# langchain_core의 prompts 모듈에서 ChatPromptTemplate 클래스를 가져옵니다.
from langchain_core.prompts import ChatPromptTemplate
# langchain_openai 모듈에서 ChatOpenAI 클래스를 가져옵니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델의 인스턴스를 생성합니다. 이 모델은 OpenAI의 GPT 모델을 사용하여 텍스트를 생성합니다.
model = ChatOpenAI()
# "tell me a joke about {topic}" 형태의 프롬프트 템플릿을 생성합니다. {topic}은 사용자로부터 입력받은 주제로 대체됩니다.
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
# 프롬프트와 모델을 연결하여 체인을 생성합니다. 이 체인은 사용자로부터 주제를 입력받아 해당 주제에 관한 농담을 생성하는 데 사용됩니다.
chain = prompt | model